In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import shutil
from subprocess import check_call
from glob import glob

import socket

import yaml
import textwrap


import cesm_tools
import util

this_notebook = 'create-new-case.ipynb'

assert 'cheyenne' in socket.gethostname(), (
    'this notebook must be run on cheyenne login node'
)

## Check out model code and configure sandbox 

In [3]:
remote = 'git@github.com:ESCOMP/CESM.git'

coderoot = f"/glade/work/{os.environ['USER']}/codes"
cesmtag = 'release-cesm2.1.3'

cesmroot = cesm_tools.code_checkout(remote, coderoot, cesmtag)

Check for right tag: /glade/work/mclong/codes/release-cesm2.1.3
HEAD detached at release-cesm2.1.3
nothing to commit, working tree clean




## Set up local directory and machine-dependent paths

In [4]:
USER = os.environ["USER"]
scriptroot = os.getcwd()

machine = "cheyenne"
if machine in ["cheyenne"]:
    inputdata = "/glade/p/cesmdata/cseg/inputdata"
    scratch = f"/glade/scratch/{USER}"
    project_root = f'/glade/work/{USER}/cmip6-cplhist'
    project_data = f'{project_root}/data'
    caseroot_root = f'{project_root}/cases'
else:
    raise ValueError(f"unknown machine: {machine}")

## Setup case options

Information on compsets is [here](https://www.cesm.ucar.edu/models/cesm2/config/2.1.3/compsets.html).

In [5]:
clobber = True

PROJECT = "P93300670"

cplhist_case = "b.e21.BHIST.f09_g17.CMIP6-historical.011"

cycle_ocn_physics = True

member_id = 2

description = """
CPLHIST control case.
The 001 test was getting CO2 from the ha2x3h stream due to:
  atm_co2_opt = 'drv_diag'
In 002, changing this setting to:
  atm_co2_opt = 'constant'
"""

if cycle_ocn_physics:
    phys_cycle = '.phys_cycle'
    compset = '1850_DATM%CPLHIST_SLND_CICE_POP2%ECO%PHYS-CYCLE_DROF%CPLHIST_SGLC_WW3'
else:
    phys_cycle = ''
    compset = "1850_DATM%CPLHIST_SLND_CICE_POP2%ECO_DROF%CPLHIST_SGLC_WW3"

case = f'g.bforced{phys_cycle}{cplhist_case[1:]}.{member_id:03d}'
resolution = "f09_g17"

# initialization/forcing
run_startdate = '1850-01-01'
datm_cplhist_yr_start = 1850
datm_cplhist_yr_end = 1870

refcase = 'b.e21.B1850.f09_g17.CMIP6-piControl.001'
refdate = "0871-01-01"

# run setting
resubmit = 2
stop_n = 10
stop_option = "nyear"
rest_n = 10
rest_option = "nyear"
wallclock = "12:00:00"
queue = "regular"

# paths
caseinfo_file = f"case-info/{case}.yml"
caseroot = f"{caseroot_root}/{case}"
rundir = f"{scratch}/{case}"
archive_root = f"{scratch}/archive/{case}"

## Create case

### Determine if existing case exists, remove if `clobber=True`.

In [6]:
# ensure root directories
os.makedirs(os.path.dirname(caseroot), exist_ok=True)
os.makedirs(os.path.dirname(caseinfo_file), exist_ok=True)

if clobber:
    check_call(['rm', '-f', caseinfo_file])

create_build_case = not os.path.exists(caseinfo_file)

if create_build_case:
    with open(caseinfo_file, "w") as fid:
        yaml.dump(
            dict(
                case=case,
                description=description,
                caseroot=caseroot,
                rundir=rundir,
                archive_root=archive_root,
                compset=compset,
                resolution=resolution,
            ),
            fid,
        )
    if os.path.exists(caseroot):
        print(f"deleting {caseroot}")
        check_call(['rm', '-fr', caseroot])
    if os.path.exists(rundir):
        print(f"deleting {rundir}")
        check_call(['rm', '-fr', rundir])

deleting /glade/work/mclong/cmip6-cplhist/cases/g.bforced.phys_cycle.e21.BHIST.f09_g17.CMIP6-historical.011.002
deleting /glade/scratch/mclong/g.bforced.phys_cycle.e21.BHIST.f09_g17.CMIP6-historical.011.002


In [7]:
if create_build_case:
    check_call(
        [
            "./create_newcase",
            "--case",
            caseroot,
            "--project",
            PROJECT,
            "--machine",
            machine,
            "--res",
            resolution,
            "--compset",
            compset,
            "--run-unsupport",
        ],
        cwd=f"{cesmroot}/cime/scripts",
    )

Compset longname is 1850_DATM%CPLHIST_SLND_CICE_POP2%ECO%PHYS-CYCLE_DROF%CPLHIST_SGLC_WW3
Compset specification file is /glade/work/mclong/codes/release-cesm2.1.3/cime/../components/pop//cime_config/config_compsets.xml
Compset forcing is 1850
ATM component is  Data driven ATM  Coupler hist data set (in this mode, it is strongly recommended that the model domain and the coupler history forcing are on the same domain)
LND component is Stub land component
ICE component is Sea ICE (cice) model version 5
OCN component is POP2 Ecosystemphys cycle option
ROF component is Data runoff modelCPLHIST mode:
GLC component is Stub glacier (land ice) component
WAV component is Wave Watch
ESP component is 
Pes     specification file is /glade/work/mclong/codes/release-cesm2.1.3/cime/../components/pop//cime_config/config_pes.xml
Compset specific settings: name is PRERUN_SCRIPT and value is $SRCROOT/components/pop/cime_config/phys_cycle_prerun
Compset specific settings: name is POSTRUN_SCRIPT and value i

## Configure case

### Apply XML settings

Documentation is provided [here](https://www.cesm.ucar.edu/models/cesm2/settings/2.1.3/).

In [8]:
def xmlchange(arg):
    """call xmlchange"""
    check_call(["./xmlchange", arg], cwd=caseroot)

In [9]:
if create_build_case:

    # forcing/forcing collections
    xmlchange("OCN_COUPLING=full")

    if not cycle_ocn_physics:
        xmlchange("OCN_TRANSIENT=1850-2000")
        xmlchange("DATM_CO2_TSERIES=omip") # this might not be necessary as 
                                           # CO2 is in the cplhist ha2x3h files
                                           # TODO: test setting atm_co2_opt = 'drv_diag'
                                           #       and eliminate this setting

    # cplhist settings
    xmlchange(f"DATM_CPLHIST_CASE={cplhist_case}")
    xmlchange(f"DATM_CPLHIST_DIR={util.cplhist_stage_root}/cpl_hist/{cplhist_case}/monthly")
    xmlchange(f"DATM_CPLHIST_YR_START={datm_cplhist_yr_start}")
    xmlchange(f"DATM_CPLHIST_YR_END={datm_cplhist_yr_end}")
    xmlchange(f"DATM_CPLHIST_YR_ALIGN={datm_cplhist_yr_start}")

    xmlchange(f"DROF_CPLHIST_CASE={cplhist_case}")
    xmlchange(f"DROF_CPLHIST_DIR={util.cplhist_stage_root}/cpl_hist/{cplhist_case}/monthly")
    xmlchange(f"DROF_CPLHIST_YR_START={datm_cplhist_yr_start}")
    xmlchange(f"DROF_CPLHIST_YR_END={datm_cplhist_yr_end}")
    xmlchange(f"DROF_CPLHIST_YR_ALIGN={datm_cplhist_yr_start}")

    xmlchange(f"RUN_STARTDATE={run_startdate}")

else:
    xmlchange("CONTINUE_RUN=TRUE")

xmlchange(f"RESUBMIT={resubmit}")
xmlchange(f"STOP_OPTION={stop_option}")
xmlchange(f"STOP_N={stop_n}")
xmlchange(f"REST_OPTION={rest_option}")
xmlchange(f"REST_N={rest_n}")
xmlchange(f"JOB_WALLCLOCK_TIME={wallclock}")
xmlchange(f"JOB_QUEUE={queue}");

## Run setup

In [10]:
if create_build_case:
    check_call(['./case.setup'], cwd=caseroot)

Setting resource.RLIMIT_STACK to -1 from (307200000, -1)
job is case.run USER_REQUESTED_WALLTIME 12:00:00 USER_REQUESTED_QUEUE regular WALLTIME_FORMAT %H:%M:%S
Creating batch scripts
Writing case.run script from input template /glade/work/mclong/codes/release-cesm2.1.3/cime/config/cesm/machines/template.case.run
Creating file .case.run
Writing case.st_archive script from input template /glade/work/mclong/codes/release-cesm2.1.3/cime/config/cesm/machines/template.st_archive
Creating file case.st_archive


/glade/work/mclong/cmip6-cplhist/cases/g.bforced.phys_cycle.e21.BHIST.f09_g17.CMIP6-historical.011.002/env_mach_specific.xml already exists, delete to replace


Creating user_nl_xxx files for components and cpl
If an old case build already exists, might want to run 'case.build --clean' before building
You can now run './preview_run' to get more info on how your case will be run


## Namelist settings

### Write namelist modifcations

Documentation for the POP namelist is [here](https://www.cesm.ucar.edu/models/cesm2/settings/2.1.3/pop2_nml.html).

In [11]:
salinity_restoring_data_file = None
if not cycle_ocn_physics:
    salinity_restoring_data_file = glob(
        f"{util.restoring_data_stage_root}/{cplhist_case}/{cplhist_case}.SSS.day.*.ieeer8"
    )

    assert len(salinity_restoring_data_file) == 1, (
        f"glob for: {util.restoring_data_stage_root}/{cplhist_case}/{cplhist_case}.SSS.day.*.ieeer8",
        f"returned unexepected number of files: {len(salinity_restoring_data_file)}"

    )
    salinity_restoring_data_file = salinity_restoring_data_file[0]

salinity_restoring_data_file

In [12]:
def list_to_nml_list(lst):
    return ",".join([f"'{v}'" for v in lst])

user_nl = dict()

if cycle_ocn_physics:
    user_nl['pop'] = textwrap.dedent(
        f"""\
    &ecosys_forcing_data_nml
      atm_co2_opt = 'const'
      atm_co2_const = 284.7
      atm_alt_co2_const = 284.7
    """
    )
else:    
    # add salinity restoring details to namelist
    user_nl['pop'] = textwrap.dedent(
        f"""\
    &ecosys_forcing_data_nml
      atm_alt_co2_opt = 'const'
      atm_alt_co2_const = 284.7
    &forcing_sfwf_nml
      sfwf_data_inc = 24.0 ! file has data every 24 hours
      sfwf_data_renorm = 1.0 ! salinity restoring file did not apply scale_factor
      sfwf_data_type = 'n-hour'
      sfwf_file_fmt = 'bin'
      sfwf_filename = '{salinity_restoring_data_file}'
      sfwf_formulation = 'restoring'
      sfwf_interp_freq = 'every-timestep'
      sfwf_interp_type = 'linear'
      sfwf_restore_tau = 30.0 ! days
    """
    )

if create_build_case:
    for key, nl in user_nl.items():
        with open(f"{caseroot}/user_nl_{key}", 'w') as fid:
            fid.write(user_nl[key])

## Build the model

In [13]:
if create_build_case:
    check_call(['qcmd', '-A', PROJECT, '--', './case.build'], cwd=caseroot)

Submitting command to PBS using account P93300670:
    ./case.build

Requested custom PBS options:
    -A P93300670 

Waiting for job 5079008.chadmin1.ib0.cheyenne.ucar.edu to start ... 


Building case in directory /glade/work/mclong/cmip6-cplhist/cases/g.bforced.phys_cycle.e21.BHIST.f09_g17.CMIP6-historical.011.002
sharedlib_only is False
model_only is False
Setting resource.RLIMIT_STACK to -1 from (-1, -1)
Generating component namelists as part of build
Creating component namelists
   Calling /glade/work/mclong/codes/release-cesm2.1.3/cime/src/components/data_comps/datm/cime_config/buildnml
   ....  Obtaining DATM model domain info from first stream file: CPLHISTForcing.Solar
   Calling /glade/work/mclong/codes/release-cesm2.1.3/cime/src/components/stub_comps/slnd/cime_config/buildnml
   Calling /glade/work/mclong/codes/release-cesm2.1.3/components/cice//cime_config/buildnml
     ...buildnml calling cice buildcpp to set build time options
   Calling /glade/work/mclong/codes/release-

## Preview namelists

In [14]:
# copy restarts
check_call(
    f"cp {util.restart_stage_root}/{refcase}/{refdate}-00000/* {rundir}/run/.",
    shell=True,
)

0

In [15]:
check_call(['./preview_namelists'], cwd=caseroot)

Setting resource.RLIMIT_STACK to -1 from (307200000, -1)
Creating component namelists
   Calling /glade/work/mclong/codes/release-cesm2.1.3/cime/src/components/data_comps/datm/cime_config/buildnml
   ....  Obtaining DATM model domain info from first stream file: CPLHISTForcing.Solar
   Calling /glade/work/mclong/codes/release-cesm2.1.3/cime/src/components/stub_comps/slnd/cime_config/buildnml
   Calling /glade/work/mclong/codes/release-cesm2.1.3/components/cice//cime_config/buildnml
   Calling /glade/work/mclong/codes/release-cesm2.1.3/components/pop//cime_config/buildnml
   Calling /glade/work/mclong/codes/release-cesm2.1.3/cime/src/components/data_comps/drof/cime_config/buildnml
   ....  Obtaining DROF model domain info from first stream file: rof.cplhist
   Calling /glade/work/mclong/codes/release-cesm2.1.3/cime/src/components/stub_comps/sglc/cime_config/buildnml
   Calling /glade/work/mclong/codes/release-cesm2.1.3/components/ww3//cime_config/buildnml
   Calling /glade/work/mclong/c

0

## Submit the job

In [16]:
check_call(['./case.submit'], cwd=caseroot)

Setting resource.RLIMIT_STACK to -1 from (307200000, -1)
Creating component namelists
   Calling /glade/work/mclong/codes/release-cesm2.1.3/cime/src/components/data_comps/datm/cime_config/buildnml
   ....  Obtaining DATM model domain info from first stream file: CPLHISTForcing.Solar
   Calling /glade/work/mclong/codes/release-cesm2.1.3/cime/src/components/stub_comps/slnd/cime_config/buildnml
   Calling /glade/work/mclong/codes/release-cesm2.1.3/components/cice//cime_config/buildnml
   Calling /glade/work/mclong/codes/release-cesm2.1.3/components/pop//cime_config/buildnml
   Calling /glade/work/mclong/codes/release-cesm2.1.3/cime/src/components/data_comps/drof/cime_config/buildnml
   ....  Obtaining DROF model domain info from first stream file: rof.cplhist
   Calling /glade/work/mclong/codes/release-cesm2.1.3/cime/src/components/stub_comps/sglc/cime_config/buildnml
   Calling /glade/work/mclong/codes/release-cesm2.1.3/components/ww3//cime_config/buildnml
   Calling /glade/work/mclong/c

submit_jobs case.run
Submit job case.run


Submitted job id is 5079030.chadmin1.ib0.cheyenne.ucar.edu
Submitting job script qsub -q regular -l walltime=12:00:00 -A P93300670  -W depend=afterok:5079030.chadmin1.ib0.cheyenne.ucar.edu -m ea -v ARGS_FOR_SCRIPT='--resubmit' case.st_archive


Submit job case.st_archive


Submitted job id is 5079031.chadmin1.ib0.cheyenne.ucar.edu
Submitted job case.run with id 5079030.chadmin1.ib0.cheyenne.ucar.edu
Submitted job case.st_archive with id 5079031.chadmin1.ib0.cheyenne.ucar.edu


0

### Copy this notebook to create record of case creation

In [18]:
shutil.copyfile(this_notebook, f"case-notebooks/{case}.ipynb")

'case-notebooks/g.bforced.phys_cycle.e21.BHIST.f09_g17.CMIP6-historical.011.002.ipynb'